In [37]:
#!pip install tensorflow

import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

In [2]:
# Carregar o conjunto de dados NSL-KDD

!wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz
!gzip -d kddcup.data_10_percent.gz


--2024-02-08 00:42:57--  http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2144903 (2.0M) [application/x-gzip]
Saving to: ‘kddcup.data_10_percent.gz’

kddcup.data_10_perc 100%[===================>]   2.04M  1.86MB/s    in 1.1s    

2024-02-08 00:42:58 (1.86 MB/s) - ‘kddcup.data_10_percent.gz’ saved [2144903/2144903]



In [38]:
df = pd.read_csv('/content/KDDTrain+.txt')

In [39]:
# Definir as colunas
columns = np.array([
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
    'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in',
    'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
    'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
    'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate',
    'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
    'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
    'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate',
    'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'label'
])

In [40]:
# Carregar os dados
#data = np.genfromtxt('KDDTrain+.txt', delimiter=',')
data = np.genfromtxt('/content/KDDTrain+.txt', delimiter=',')
X = data[:, :-1]
y_str = data[:, -1].astype(object)

In [43]:
data

array([[ 0., nan, nan, ...,  0., nan, 20.],
       [ 0., nan, nan, ...,  0., nan, 15.],
       [ 0., nan, nan, ...,  0., nan, 19.],
       ...,
       [ 0., nan, nan, ...,  0., nan, 18.],
       [ 0., nan, nan, ...,  0., nan, 20.],
       [ 0., nan, nan, ...,  0., nan, 21.]])

In [44]:
# Pré-processamento dos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [45]:
# Codificação one-hot para variáveis categóricas
X_encoded = np.zeros((X.shape[0], X.shape[1] + 3))
for i in range(X.shape[1]):
    if i not in [1, 2, 3]:  # As colunas 1, 2 e 3 são variáveis categóricas
        X_encoded[:, i] = X_scaled[:, i]
    else:
        unique_values = np.unique(X[:, i])
        for j, value in enumerate(unique_values):
            X_encoded[:, j + X.shape[1]] = (X[:, i] == value).astype(int)

In [46]:
# Codificar os rótulos
labels = np.unique(y_str)
y_encoded = np.array([np.where(labels == label)[0][0] for label in y_str])

In [47]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

In [48]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [49]:
# Definir a arquitetura do modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(labels), activation='softmax')
])

In [50]:
# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [36]:
print(X_test.shape)
print(y_test.shape)

(25195, 45)
(25195,)


In [56]:
# Treinar o modelo localmente
model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/25
3150/3150 [==============================] - 4s 1ms/step - loss: nan - accuracy: 4.9614e-04 - val_loss: nan - val_accuracy: 6.3505e-04
Epoch 2/25
3150/3150 [==============================] - 5s 2ms/step - loss: nan - accuracy: 4.9614e-04 - val_loss: nan - val_accuracy: 6.3505e-04
Epoch 3/25
3150/3150 [==============================] - 4s 1ms/step - loss: nan - accuracy: 4.9614e-04 - val_loss: nan - val_accuracy: 6.3505e-04
Epoch 4/25
3150/3150 [==============================] - 5s 1ms/step - loss: nan - accuracy: 4.9614e-04 - val_loss: nan - val_accuracy: 6.3505e-04
Epoch 5/25
3150/3150 [==============================] - 5s 1ms/step - loss: nan - accuracy: 4.9614e-04 - val_loss: nan - val_accuracy: 6.3505e-04
Epoch 6/25
3150/3150 [==============================] - 5s 1ms/step - loss: nan - accuracy: 4.9614e-04 - val_loss: nan - val_accuracy: 6.3505e-04
Epoch 7/25
3150/3150 [==============================] - 5s 2ms/step - loss: nan - accuracy: 4.9614e-04 - val_loss: nan - val

In [52]:
# Função para dividir os dados em shards
def create_shards(X, y, num_clients):
    X_shards, y_shards = [], []
    X, y = shuffle(X, y)
    shard_size = len(X) // num_clients
    for i in range(num_clients):
        start = i * shard_size
        end = (i + 1) * shard_size if i < num_clients - 1 else len(X)
        X_shards.append(X[start:end])
        y_shards.append(y[start:end])
    return X_shards, y_shards

In [53]:

# Configurações do Federated Learning
num_clients = 5
num_rounds = 10
batch_size = 32

In [54]:
# Dividir os dados em shards para os clientes
X_shards, y_shards = create_shards(X_train, y_train, num_clients)

In [55]:

# Iniciar o treinamento federado
for round_num in range(num_rounds):
    print(f"Round {round_num + 1}/{num_rounds}")
    for client in range(num_clients):
        print(f"\tClient {client + 1}/{num_clients}")
        model_weights = model.get_weights()
        client_X, client_y = X_shards[client], y_shards[client]
        client_dataset = tf.data.Dataset.from_tensor_slices((client_X, client_y)).shuffle(len(client_X)).batch(batch_size)
        client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
        for X_batch, y_batch in client_dataset:
            with tf.GradientTape() as tape:
                y_pred = model(X_batch)
                loss = tf.keras.losses.sparse_categorical_crossentropy(y_batch, y_pred)
            gradients = tape.gradient(loss, model.trainable_variables)
            client_optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    print("\tEvaluating global model")
    print(f"\tRound {round_num + 1} Accuracy:", model.evaluate(X_test, y_test, verbose=0)[1])

print("Federated Learning Finished")


Round 1/10
	Client 1/5
	Client 2/5
	Client 3/5
	Client 4/5
	Client 5/5
	Evaluating global model
	Round 1 Accuracy: 0.0006350466283038259
Round 2/10
	Client 1/5
	Client 2/5
	Client 3/5


KeyboardInterrupt: 